In [21]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags, csc_matrix
from scipy.sparse.linalg import spsolve

# Parameters
L = 35  # Domain size (in dimensionless units)
dx = 0.006  # Spatial step
N = int (L / dx)
print(N)
xi = np.linspace(-L / 2, L / 2, N)  # Spatial grid in dimensionless units
dt = 0.001  # Temporal step (in dimensionless units)
t_max = 15 # Final time (maximum tuning time)
t_steps = int(t_max / dt)  # Number of time steps

# Wave packet parameters, initial wave function
xi0 = -4 # Initial position of the particle
kappa_0 = 4 # Initial momentum
psi0 = (2 / np.pi)**(1/4) * np.exp(-(xi - xi0)**2) * np.exp(1j * kappa_0 * xi)

# Potential barrier
alpha_param = 6  
v_0 = 0 # Height of the potential (V_0/Ec)
b = 1 # Thickness of the barrier (in dimensionless units, really xi_b)
v = v_0 / (1 + np.abs(xi / b)**alpha_param)  # Explicit form of the potential

# Crank-Nicolson matrices 
alpha = 1j * dt / (kappa_0 * dx**2)  
beta = 1j * dt * kappa_0 / 2  

print(alpha)

diagonal_a = (1 + alpha + beta * v) * np.ones(N)
off_diagonal = -(alpha / 2) * np.ones(N - 1)
A = csc_matrix(diags([off_diagonal, diagonal_a, off_diagonal], [-1, 0, 1]))  # Convert to CSC format

diagonal_b = (1 - alpha - beta * v) * np.ones(N)
B = csc_matrix(diags([-off_diagonal, diagonal_b, -off_diagonal], [-1, 0, 1]))  # Convert to CSC format

def spectral_radius(A, B):
    n = A.shape[0]
    
    # Calculate A^(-1) * B
    A_inv_B = spsolve(A, B)  # Efficiently computes A^(-1) * B for sparse matrices
    
    # Eigenvalues can be approximated by the Rayleigh quotient
    eigenvalues = []
    
    for i in range(n):
        b = np.zeros(n)
        b[i] = 1  # Standard basis vector
        x = A_inv_B @ b  # This computes A^(-1) * B * e_i
        eigenvalues.append(np.dot(b, x))  # Compute the Rayleigh quotient
    
    # Compute the spectral radius
    spectral_rad = max(abs(np.array(eigenvalues)))
    return spectral_rad

rho = spectral_radius(A, B)

print("Spectral radius of A^{-1}B:", rho)





5833
6.944444444444445j


KeyboardInterrupt: 